# "Fallecidos DEIS por comuna"
> Fallecidos DEIS por comuna.

- toc: true 
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/diagram.png

In [1]:
#hide
import numpy as np
import pandas as pd
import altair as alt
from IPython.display import display, Markdown, display_html, HTML

In [2]:
#hide
update_date = pd.to_datetime('today') - pd.offsets.Hour(19)

In [3]:
#hide_input
display(Markdown(f"Última actualización: {update_date.strftime('%d/%m/%Y')}."))

Última actualización: 30/06/2020.

# Nuevas defunciones causas Covid Confirmado y Covid Sospecho por comuna de la Región Metropolitana

In [4]:
#hide
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/alonsosilvaallende/COVID-19/master/data/DEIS_causa_COVID.csv")

In [5]:
#hide
data_raw = data_raw.drop(columns="año")

In [6]:
#hide
data_RM = data_raw.query("region=='Metropolitana de Santiago'")

In [7]:
#hide
county_deaths_RM = pd.DataFrame()
county_deaths_RM["Fecha"] = pd.date_range(start="2020-03-16", end="2020-06-25")
county_deaths_RM = county_deaths_RM.set_index("Fecha")

In [8]:
#hide
for comuna in data_RM["comuna"].unique():
  aux = data_raw[data_raw["comuna"] == f"{comuna}"]
  deaths = pd.Series(dtype='float64')
  for date in aux["fecha"].unique():
    deaths[f'{date}'] = len(aux.query(f"fecha == '{date}'"))
  deaths.index = pd.to_datetime(deaths.index)
  county_deaths_RM[f'{comuna}'] = deaths

In [24]:
#hide
data_long_form = county_deaths_RM.reset_index().melt("Fecha", var_name="Comuna", value_name = "Fallecidos confirmados")

In [23]:
#hide
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [22]:
#hide_input
input_dropdown = alt.binding_select(options=data_long_form['Comuna'].unique())
selection1 = alt.selection_single(fields=['Comuna'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Comuna'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Comuna:N', scale=alt.Scale(scheme='category20'), legend=None),
                    alt.value('lightgray'))

bars1 = alt.Chart(data_long_form).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False, tickSize=0)),
    y=alt.Y("Fallecidos confirmados", axis=alt.Axis(title="número de defunciones")),
    tooltip = ["Fecha", "Comuna", "Fallecidos confirmados"],
    color = color
).properties(
    title="Nuevas defunciones causas COVID confirmado y COVID sospechoso por comuna de la R.M.",
    width=700).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

bars1

alt.Chart(...)

# Fallecidos DEIS por comuna

In [25]:
#hide
confirmed_deaths = pd.Series(dtype="float64")
suspected_deaths = pd.Series(dtype='float64')
for comuna in data_RM["comuna"].unique():
    confirmed_deaths[f"{comuna}"] = len(data_raw[(data_raw["código_detalle"] == 'U07.1') & (data_raw["comuna"] == f'{comuna}')])
    suspected_deaths[f"{comuna}"] = len(data_raw[(data_raw["código_detalle"] == 'U07.2') & (data_raw["comuna"] == f'{comuna}')])

In [26]:
#hide
deaths = pd.DataFrame()
deaths["Comuna"] = data_RM["comuna"].unique()
deaths = deaths.set_index("Comuna")

In [27]:
#hide
deaths["COVID Confirmado"] = confirmed_deaths
deaths["COVID Sospechoso"] = suspected_deaths

In [28]:
#hide
deaths_long_format = deaths.reset_index().melt("Comuna", var_name="Causa", value_name="Fallecidos")

In [29]:
#hide_input
bars = alt.Chart(deaths_long_format).mark_bar(opacity=0.7, size=25).encode(
    x = alt.X('Fallecidos:Q', stack='zero'),
    y = alt.Y('Comuna', sort='-x'),
    tooltip = ['Comuna', 'Fallecidos'],
    color = alt.Color('Causa', sort=['Confirmado','Sospechoso']),
    order = alt.Order('Causa')
)

text = alt.Chart(deaths_long_format).mark_text(dx=-11, dy=0, color="#636363").encode(
     x=alt.X('Fallecidos:Q', stack='zero'),
     y=alt.Y('Comuna', sort='-x'),
     color=alt.Color('Causa', sort=['Confirmado','Sospechoso']),
     order = alt.Order("Causa"),
     text=alt.Text("Fallecidos", format=',.0f')
)
(bars+text).properties(
    title = f"Total fallecidos DEIS por comuna de la R.M. al {pd.to_datetime('2020-06-28').strftime('%d/%m')}",
    width = 600,
    height = alt.Step(30)
)

alt.LayerChart(...)